In [ ]:
import pickle
import json
files = "./CasRel/data/baidu/duie_train.json"
with open(files,"rb") as f:
    data = f.readlines()
index = 0
a = ['上映时间', '专业代码', '改编自', '修业年限', '成立日期', 'ondate_date', '董事长', '注册资本', '占地面积', '人口数量', '所属专辑', 'inwork_影视作品', '编剧', '歌手', '票房', '导演', '主演', 'period_number', '邮政编码', '作者', '简称', '面积', '作曲', '作词']
predicates = set()
max_leng = 0
for line in data:
    line = json.loads(line)
    spo_list = line['spo_list']
    text = line['text']
    max_leng = max(len(text),max_leng)
    for spo in spo_list:
        s = spo['subject']
        p = spo['object']['@value']
        predicate = spo['predicate']
        if ( s.isdigit() or p.isdigit() ) and predicate == a[1]:
            # print(p+"\t"+predicate+"\t"+s+"\t"+text)
            predicates.add(predicate)
            index += 1
# print(predicates)
print(max_leng)


In [ ]:
import numpy as np
import torch


aa = np.array([[0.0349,  0.0670, -0.0612, 0.0280, -0.0222,  0.0422],
         [-1.6719,  0.1242, -0.6488, 0.3313, -1.3965, -0.0682],
         [-1.3419,  0.4485, -0.6589, 0.1420, -0.3260, -0.4795]])
bb = np.array([[-0.0658, -0.1490, -0.1684, 0.7188,  0.3129, -0.1116],
         [-0.2098, -0.2980,  0.1126, 0.9666, -0.0178,  0.1222],
         [ 0.1179, -0.4622, -0.2112, 1.1151,  0.1846,  0.4283]])
cc = np.where(aa>0)     #合并a,b两个tensor，如果a中元素大于0
dd = np.where(bb>0)


In [ ]:
import torch
sub_heads = torch.tensor([2,3,2,5,2,7,6,7,8,2])
sub_tails = torch.tensor([5,7,5,2,7,3,8,7,8,7])
sub_head_mapping = torch.zeros(10,1,9)
sub_tail_mapping = torch.zeros(10,1,9)
sub_head_mapping.scatter_(-1,sub_heads.reshape(10,1,1),1)
sub_tail_mapping.scatter_(-1,sub_tails.reshape(10,1,1),1)

In [7]:
from transformers import BertTokenizer
text = "内地影视演员龚蓓苾携手周渝民、迪丽热巴、张彬彬、刘芮麟、赖艺、代斯、张赫等主创悉数出席"
tokenizer = BertTokenizer.from_pretrained("../chinese_bert_base")
# print(tokenizer.tokenize(text))
# token = ['龚', '蓓', '苾', '携', '手', '周', '渝', '民', '、', '迪', '丽', '热', '巴', '、', '张', '彬', '彬', '、', '刘', '芮', '麟', '、', '赖', '艺', '、', '代', '斯', '、', '张', '赫', '等', '主', '创', '悉', '数', '出', '席']
# tokenizer.convert_tokens_to_ids(token)

In [8]:
import torch
head_size = 50
heads=4
a = torch.rand(10,30,head_size* heads * 2)
input1 = torch.split(a,head_size*2,dim = -1)
print(input1[0].shape)
input2 = torch.stack(input1, axis=-2)
print(input2.shape)
qw, kw = input2[..., :head_size], input2[..., head_size:]
print(qw.shape)
print(qw[..., 1::2].shape)
qw2 = torch.stack([-qw[..., 1::2], qw[..., ::2]], 4)
print(qw2.shape)
qw2 = torch.reshape(qw2, qw.shape)
print(qw2.shape)

torch.Size([10, 30, 100])
torch.Size([10, 30, 4, 100])
torch.Size([10, 30, 4, 50])
torch.Size([10, 30, 4, 25])
torch.Size([10, 30, 4, 25, 2])
torch.Size([10, 30, 4, 50])


In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
inputs = tf.random_normal((10,20,head_size* heads * 2))
print(inputs.shape)

(10, 20, 400)


In [41]:
input1 = tf.split(inputs, heads, axis=-1)
print(len(input1))
print(input1[0].shape)
input2 = K.stack(input1, axis=-2)
print(input2.shape)
qw, kw = input2[..., :head_size], input2[..., head_size:]
print(qw.shape)
qw2 = K.stack([-qw[..., 1::2], qw[..., ::2]], 4)
print(qw2.shape)
qw2 = K.reshape(qw2, K.shape(qw))
print(qw2.shape)


4
(10, 20, 100)
(10, 20, 4, 100)
(10, 20, 4, 50)
(10, 20, 4, 25, 2)
(10, 20, 4, 50)


In [81]:
inputs = torch.rand((10,100,20))
output_dim = 50
position_ids = torch.arange(0, 100, dtype=torch.float).reshape(1,-1)
indices = torch.arange(0, output_dim // 2, dtype=torch.float)
indices = torch.pow(10000.0, -2 * indices / output_dim).reshape(1,-1)
embeddings = torch.matmul(position_ids.unsqueeze(-1),indices.unsqueeze(0))
embeddings = torch.stack([torch.sin(embeddings), torch.cos(embeddings)], axis=-1)
# embeddings = torch.reshape(embeddings, (-1, 100, output_dim))
print(embeddings.shape)
# embeddings = embeddings.repeat([10, 1, 1])
# torch.cat([inputs, embeddings],axis=-1).shape

torch.Size([1, 100, 25, 2])


In [76]:
inputs = tf.random_normal((10,100,20))
position_ids = K.arange(0, 100, dtype=K.floatx())[None]

indices = K.arange(0, output_dim // 2, dtype=K.floatx())
indices = K.pow(10000.0, -2 * indices / output_dim)
print(indices.shape)
embeddings = tf.einsum('bn,d->bnd', position_ids, indices)
embeddings = K.stack([K.sin(embeddings), K.cos(embeddings)], axis=-1)
embeddings = K.reshape(embeddings, (-1, 100, output_dim))

print(embeddings.shape)
embeddings = K.tile(embeddings, [10, 1, 1])
K.concatenate([inputs, embeddings])


(25,)
(1, 100, 50)


<tf.Tensor 'concat:0' shape=(10, 100, 70) dtype=float32>

In [9]:
qw = torch.rand(10,30,2,20)
kw = torch.rand(10,30,2,20)
# torch.matmul(qw.transpose(2,1),kw.permute(0,3,1,2))
print(qw.transpose(2,1).shape)
kww = kw.transpose(2,1)
print(kww.shape)
print(kww.transpose(3,2).shape)
print(kw.permute(0,2,3,1).shape)
kww.transpose(3,2) == kw.permute(0,2,3,1)

torch.Size([10, 2, 30, 20])
torch.Size([10, 2, 30, 20])
torch.Size([10, 2, 20, 30])
torch.Size([10, 2, 20, 30])


tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]],


        [[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  .

In [6]:
x = tf.constant([[0., 1., 2.], [0., 1., 2.]])
y1 = tf.reduce_logsumexp(x)             # 3.1007533
y2 = tf.reduce_logsumexp(x, 0)

sess = tf.Session()
# Evaluate the tensor `c`.

print(sess.run(y1))
print(sess.run(y2))
x = torch.tensor([[0., 1., 2.], [0., 1., 2.]])
y2 = torch.logsumexp(x, dim=0)
print(y2)

3.1007533
[0.6931472 1.6931472 2.6931472]


In [6]:
import numpy as np
np.where(np.random.rand(5,10,10)>0.5)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int64),
 array([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
        4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9,
        9, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 